In [2]:
import pandas as pd
import tensorflow as tf
import datetime

In [3]:
import numpy as np

In [4]:
np.random.seed(42)
tf.random.set_seed=42




In [5]:
import json
with open('data_full.json') as json_file:
    CLINC150 = json.load(json_file)
CLINC150_train=CLINC150['train']
CLINC150_test=CLINC150['test']
CLINC150_val=CLINC150['val']


In [6]:
classes=['insurance',
 'next_holiday',
 'repeat',
 'credit_limit_change',
 'book_hotel',
 'yes',
 'damaged_card',
 'rewards_balance',
 'time',
 'pto_balance',
 'interest_rate',
 'change_volume',
 'taxes',
 'sync_device',
 'traffic',
 'what_song',
 'shopping_list',
 'todo_list_update',
 'order_checks',
 'shopping_list_update']

In [ ]:
train_data=[]

In [ ]:
for c in CLINC150_train:
    if c[1] in classes:
        train_data.append(c)

In [ ]:
df = pd.DataFrame(train_data)
df.to_csv('train_data.csv', index=False,header=('text','intent'))
train=pd.read_csv('train_data.csv')
print(len(train))
train.head()

2000


,text,intent
0,"what time is it in punta gorda, florida",time
1,"what time is it in glenwood springs, co",time
2,"what time is it in fredericksburg, tx",time
3,"what time is it in las vegas, nv",time
4,"what time is it in houston, tx",time


In [7]:
train=pd.read_csv('train_data.csv')
print(len(train))

2000


In [8]:
train.shape

(2000, 2)

In [9]:
val_data=[]


In [24]:
for c in CLINC150_val:
    if c[1] in classes:
        val_data.append(c)

In [ ]:
df = pd.DataFrame(val_data)
df.to_csv('val_data.csv', index=False,header=('text','intent'))
val=pd.read_csv('train_data.csv')
print(len(val))
val.head(20)

In [10]:
import tensorflow_hub as hub
import tensorflow_text as text

In [11]:
bert_process=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4")

In [12]:
X=train['text'].values
y=train['intent'].values


In [13]:
y

array(['time', 'time', 'time', ..., 'order_checks', 'order_checks',
       'order_checks'], dtype=object)

In [14]:
y_train=[]
for i in y:
    y_train.append(classes.index(i))

In [15]:
y_train=np.array(y_train)

In [16]:
y_train

array([ 8,  8,  8, ..., 18, 18, 18])

In [17]:
from tensorflow.keras.layers import Input,Dropout,Dense

In [18]:
inputs=Input(shape=(),dtype=tf.string,name='text')
preprocessed_text=bert_process(inputs)
encoder=bert_encoder(preprocessed_text)


#drop=Dropout(0.2,name='dropout')(encoder['pooled_output'])
cls_out=Dense(768,activation='relu')(encoder['pooled_output'])
drop=Dropout(0.2)(cls_out)
output=Dense(units=len(classes),activation='softmax',name='outut')(drop)


In [19]:
from tensorflow.keras.models import Model

In [20]:
model=Model(inputs=[inputs],outputs=[output])


In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [22]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [23]:
y_train[0].shape

()

In [24]:
log_dir = "log/intent_detection/" + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[-3]
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


In [25]:
histpory=model.fit(X,y_train,epochs=30,batch_size=16,validation_split=0.2,shuffle=True,callbacks=[tensorboard_callback])


Epoch 1/30


c:\Users\Niranjan Hegde\bert\bert\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


100/100 [==============================] - 77s 617ms/step - loss: 2.4247 - sparse_categorical_accuracy: 0.2525 - val_loss: 9.5232 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/30
100/100 [==============================] - 67s 671ms/step - loss: 1.5855 - sparse_categorical_accuracy: 0.5881 - val_loss: 10.2759 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/30
100/100 [==============================] - 69s 686ms/step - loss: 1.0675 - sparse_categorical_accuracy: 0.7237 - val_loss: 11.1548 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/30
100/100 [==============================] - 67s 672ms/step - loss: 0.7520 - sparse_categorical_accuracy: 0.8219 - val_loss: 11.8189 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/30
100/100 [==============================] - 69s 690ms/step - loss: 0.5858 - sparse_categorical_accuracy: 0.8531 - val_loss: 12.3846 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/30
  7/100 [=>............................] - ETA: 58s - loss: 0

KeyboardInterrupt: 

In [85]:
sentences = [
  "wie spät ist es in london",
  "ಫೀನಿಕ್ಸ್‌ನಲ್ಲಿ ಉತ್ತಮ ವಿಮರ್ಶೆಗಳನ್ನು ಹೊಂದಿರುವ ಹೋಟೆಲ್ ಅನ್ನು ನನಗೆ ಹುಡುಕಿ",
  "what is song in swadesh",
  "rush",
  "merci de m'envoyer d'autres chéquiers"
]

predictions = model.predict(sentences)

1/1 [==============================] - 0s 162ms/step


In [86]:
predictions.argmax(axis=-1)

array([ 8,  4, 15,  5, 13], dtype=int64)

In [87]:
classes[13]

'sync_device'

In [50]:
val=pd.read_csv('val_data.csv')

In [51]:
val

,text,intent
0,what time is it in france,time
1,what's the time in london right now,time
2,what hour is it in london,time
3,what's the time,time
4,what is the time in london,time
...,...,...
395,please send me more checkbooks,order_checks
396,please notify my bank that i need more checks,order_checks
397,how can i order checks for my savings account,order_checks
398,i need to know how to order checks for my bank...,order_checks


In [52]:
sentences=val['text'].values
y_test=val['intent'].values

In [53]:
y_test_=[]

In [54]:
for i in y_test:
    y_test_.append(classes.index(i))

In [55]:
y_test_=np.array(y_test_)

In [56]:
y_pred=model.predict(sentences).argmax(axis=-1)

13/13 [==============================] - 11s 922ms/step


In [57]:
y_pred

array([ 8,  8,  8, 15,  8,  8,  8, 15,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  5, 19, 19, 19, 19, 19, 17, 19, 17, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 10,  7, 10, 10,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  2,  2,  5,  2,  2,  2,  1,  1,
        2,  5,  5,  2,  2,  2,  2,  1,  2,  2,  1,  2,  5,  5,  5,  5,  5,
       12,  5,  5,  5,  5,  8, 17, 17,  5,  5,  5,  5,  5,  5,  5,  0,  0,
        0,  0,  0,  9,  0,  4,  0,  4,  4, 10, 10,  0,  0, 12,  9,  0,  0,
        0, 17, 17, 17, 17, 17, 17, 17, 17, 19, 17, 17, 17, 17, 17, 17, 17,
       19, 17, 17, 17, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  1,  6,  6,  6,  1,  6,  6,  6,  6,  8,  1,  1,  1,  1,  1,  1,
        1,  9,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 15,
       15, 15, 15,  1, 15

In [58]:
y_test_

array([ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 15,
       15, 15, 15, 15, 15

In [61]:
from sklearn.metrics import confusion_matrix


AttributeError: module 'scipy' has no attribute '_lib'

In [ ]:
print(confusion_matrix(y_test_,y_pred))

In [167]:
test_data=[]
for c in CLINC150_test:
    if c[1] in classes:
        test_data.append(c)


In [168]:
df = pd.DataFrame(test_data)
df.to_csv('test_data.csv', index=False,header=('text','intent'))
test=pd.read_csv('test_data.csv')
print(len(test))
test.head()

600


,text,intent
0,i need you to tell me what time it is in new y...,time
1,"what time is it in adelaide, australia right now",time
2,is it after noon,time
3,is it six o clock yet,time
4,please give me the time in tanzania at this mo...,time


In [169]:
X_test_=test['text'].values

In [170]:
y_test__=test['intent'].values

In [171]:
y_test_actual=[]

In [172]:

for i in y_test__:
    y_test_actual.append(classes.index(i))

In [173]:
y_test_actual=np.array(y_test_actual)

In [174]:
predictions_test=model.predict(X_test_).argmax(-1)

19/19 [==============================] - 18s 979ms/step


In [175]:
predictions_test

array([ 8,  8,  8,  8,  2,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  4,
        8, 15,  8,  8,  8, 15,  8,  8,  8,  8,  8,  8,  8, 19, 17, 19, 15,
       19, 19, 19, 19, 19, 19, 19, 17, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 17, 17, 19,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  6,  0,  7,
        7,  7,  7,  7,  7,  2,  2,  2,  2,  2,  2,  2, 11,  5,  2,  5,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 11,
        2,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5, 11,  5,  5,  5,  5,  5,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17,  2, 17, 17, 17, 17, 13, 13, 13, 13, 13,  6, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13

In [176]:
classes[2]

'repeat'

In [177]:
print(classification_report(y_test_actual,predictions_test))

              precision    recall  f1-score   support

           0       0.56      1.00      0.71        30
           1       0.80      0.93      0.86        30
           2       0.90      0.87      0.88        30
           3       0.00      0.00      0.00        30
           4       0.64      1.00      0.78        30
           5       0.81      0.97      0.88        30
           6       0.63      0.97      0.76        30
           7       0.52      0.93      0.67        30
           8       0.81      0.87      0.84        30
           9       0.86      0.80      0.83        30
          10       0.81      0.73      0.77        30
          11       0.89      0.83      0.86        30
          12       0.76      0.97      0.85        30
          13       0.91      0.97      0.94        30
          14       0.00      0.00      0.00        30
          15       0.72      0.93      0.81        30
          16       0.00      0.00      0.00        30
          17       0.78    

c:\Users\Niranjan Hegde\bert\bert\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Niranjan Hegde\bert\bert\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Niranjan Hegde\bert\bert\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [133]:
model.save('')

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


In [88]:
df=pd.read_csv('train.csv')

In [89]:
df

,Unnamed: 0,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
0,103626,1,en-US,train,alarm,alarm_set,wake me up at nine am on friday,wake me up at [time : nine am] on [date : friday],1,NaN,NaN
1,103627,2,en-US,train,alarm,alarm_set,set an alarm for two hours from now,set an alarm for [time : two hours from now],1,NaN,NaN
2,103628,4,en-US,train,audio,audio_volume_mute,olly quiet,olly quiet,1,NaN,NaN
3,103629,5,en-US,train,audio,audio_volume_mute,stop,stop,1,NaN,NaN
4,103630,6,en-US,train,audio,audio_volume_mute,olly pause for ten seconds,olly pause for [time : ten seconds],1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11509,115135,17175,en-US,train,email,email_querycontact,send hi in watsapp to vikki,send hi in watsapp to vikki,11,NaN,NaN
11510,115136,17176,en-US,train,email,email_query,do i have emails,do i have emails,12,NaN,NaN
11511,115137,17177,en-US,train,email,email_query,what emails are new,what emails are new,12,NaN,NaN
11512,115138,17178,en-US,train,email,email_query,do i have new emails from john,do i have new emails from [person : john],12,NaN,NaN


In [90]:
utt=df['utt'].values

In [98]:
df['intent'].value_counts()

calendar_set                810
play_music                  639
weather_query               573
calendar_query              566
general_quirky              555
qa_factoid                  544
news_query                  503
email_query                 418
email_sendemail             354
datetime_query              350
calendar_remove             312
play_radio                  283
social_post                 283
qa_definition               267
transport_query             227
cooking_recipe              207
lists_query                 198
play_podcasts               193
recommendation_events       190
alarm_set                   182
lists_createoradd           177
recommendation_locations    173
lists_remove                164
music_query                 154
iot_hue_lightoff            153
qa_stock                    152
play_audiobook              150
qa_currency                 142
takeaway_order              135
alarm_query                 130
transport_ticket            127
email_qu

In [97]:
from sklearn.preprocessing import LabelEncoder

AttributeError: module 'scipy' has no attribute '_lib'